In [ ]:
f = [
# '/data/dzuberi/code/badger-bench/server/new_client/run.20240522132514-uintr-10000us/results_10000us.csv',
# '/data/dzuberi/code/badger-bench/server/new_client/run.20240522140207-signal-10000us/results_10000us.csv',
# '/data/dzuberi/code/badger-bench/server/new_client/run.20240522143902-uintr-100us/results_100us.csv',
# '/data/dzuberi/code/badger-bench/server/new_client/run.20240522151556-signal-100us/results_100us.csv'
'/data/dzuberi/code/badger-bench/server/new_client/run.20240528124453-uintr-10000us/results_10000us.csv',
'/data/dzuberi/code/badger-bench/server/new_client/run.20240528132200-signal-10000us/results_10000us.csv',
'/data/dzuberi/code/badger-bench/server/new_client/run.20240528135914-uintr-100us/results_100us.csv',
'/data/dzuberi/code/badger-bench/server/new_client/run.20240528143631-signal-100us/results_100us.csv',
'/data/dzuberi/code/badger-bench/server/new_client/run.20240528151350-uintr-10us/results_10us.csv',
'/data/dzuberi/code/badger-bench/server/new_client/run.20240528155100-signal-10us/results_10us.csv'
]

: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

: 

In [ ]:
def prep_data(file_paths):
    # Read in data
    data_frames = []
    for f in file_paths:
        print(f"reading data from file: {f}")
        df = pd.read_csv(f, sep=",")
        df["preempt_interval_str"] = (df["preempt_interval"] / 1000).astype(int).astype(str) + "us"
 #       df["preempt_interval_str"] = pd.Categorical(df["preempt_interval_str"], categories=interval_order, ordered=True)
        df["config"] = df["preempt_type"] + " " + df["preempt_interval_str"]

 #       filtered_df = df[df["config"].str.contains("signal 10000us|signal 80us|signal 10us|uintr 10000us|uintr 50us|uintr 10us")]
 #       data_frames.append(filtered_df)
        data_frames.append(df)

    all_data = pd.concat(data_frames, ignore_index=True)

    return all_data

: 

In [ ]:
def print_columns(data):
    print("Available columns for plotting:")
    for column in data.columns:
        print(f"- {column}")

: 

In [ ]:
def plot_latency_percentile_bimodal(all_data, percentile):
    # Plot settings
    plt.figure(figsize=(5.5, 4))

    # Short request type
    data_short = all_data[all_data['req_type'] == "short"]
    max_latency_short = data_short[percentile].max()
    
    # Plot short requests
    sns.lineplot(data=data_short, x="achieved", y=percentile, hue="config", style="preempt_type", markers=True, dashes=False)
    plt.xlabel("Achieved load (req/s)")
    plt.ylabel(f"{percentile} Latency (us)")
    plt.ylim(0, max_latency_short)
    plt.legend(title="Config and Preempt Type")
    plt.title(f"Latency for {percentile} Percentile (short)")
    plt.savefig(f"latency_{percentile}_short.pdf", dpi=300)
    plt.clf()

    # Long request type
    data_long = all_data[all_data['req_type'] == "long"]
    max_latency_long = data_long[percentile].max()
    
    # Plot long requests
    sns.lineplot(data=data_long, x="achieved", y=percentile, hue="config", style="preempt_type", markers=True, dashes=False)
    plt.xlabel("Achieved load (req/s)")
    plt.ylabel(f"{percentile} Latency (us)")
    plt.ylim(0, max_latency_long)
    plt.legend(title="Config and Preempt Type")
    plt.title(f"Latency for {percentile} Percentile (long)")
    plt.savefig(f"latency_{percentile}_long.pdf", dpi=300)
    plt.clf()


: 

In [ ]:
max_latency_short = {
    "p50": 1000,
    "p99": 5000,
    "p99.9": 5000
}

max_latency_long = {
    "p50": 2000,
    "p99": 10000,
    "p99.9": 10000
}

def theme_defaults():
    return theme_bw() + theme(legend_position=(0.3, 0.7), legend_direction='vertical',
        legend_key_size=10, legend_spacing=10, legend_title=element_blank())

def plot_latency_percentile_bimodal(all_data, percentile):
    data_short = all_data[all_data['req_type'] == "short"]

    plot = ggplot(data_short, aes(x="achieved", y=percentile, color="config", linetype="preempt_type"))
    plot = plot + geom_point() + geom_line()

    plot = plot + theme_defaults()
    plot = plot + labs(x="Achieved load (req/s)", y=f"{percentile} Latency (us)")
    # plot = plot + coord_cartesian(ylim=(0, max_latency_short[percentile]))

    plot.show()
    ggsave(plot, filename=f"latency_{percentile}_short.pdf", width=5.5, height=4, dpi=300)

    data_long = all_data[all_data['req_type'] == "long"]

    plot = ggplot(data_long, aes(x="achieved", y=percentile, color="config", linetype="preempt_type"))
    plot = plot + geom_point() + geom_line()

    plot = plot + theme_defaults()
    plot = plot + labs(x="Achieved load (req/s)", y=f"{percentile} Latency (us)")
    # plot = plot + coord_cartesian(ylim=(0, max_latency_long[percentile]))

    plot.show()
    ggsave(plot, filename=f"latency_{percentile}_long.pdf", width=5.5, height=4, dpi=300)

: 

In [ ]:
all_data = prep_data(f)

print_columns(all_data)

: 

In [ ]:
plot_latency_percentile_bimodal(all_data, "p50")
plot_latency_percentile_bimodal(all_data, "p99")
plot_latency_percentile_bimodal(all_data, "p99.9")

: 

In [ ]:
import matplotlib.pyplot as plt

data_short = all_data[all_data["req_type"] == 'short']
data_long = all_data[all_data["req_type"] == 'long']

fig, axs = plt.subplots(1,2)

axs[0].scatter(data_short["achieved"], data_short["p50"], label="short")
axs[1].scatter(data_long["achieved"], data_long["p50"], label="short")

fig.set_size_inches(10,5)
fig.tight_layout()

plt.show()

: 

In [ ]:
data_short

: 

: 